<h1> Dropout aplicado en la capa full connected

In [17]:
from keras.datasets import mnist
import random
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# No necesitamos tantos datos.
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#random.seed(123) # Vamos a controlar la aleatoriedad en adelante.
X, y = zip(*random.sample(list(zip(X_train, y_train)), 2000))

# Sí necesitamos que la forma de X sea la de un vector, en lugar de una matriz.
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')
X = np.reshape(X, (X.shape[0], -1))

# Normalizamos Min-Max
X= MinMaxScaler().fit_transform(X)

# Dividimos la muestra en dos, una para entrenar y otra para testing, como tenemos
# muestra de sobra nos damos el lujo de testear con la misma cantidad que entrenamos.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

# Necesitamos que y_train sea un valor categórico, en lugar de un dígito entero.
y_train_value = y_train # Guardaremos y_train como valor para un observación más abajo.
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [18]:
import numpy as np

# Clase base para Capa
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input, training=True):
        raise NotImplementedError
    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

# Clase para capas densas (fully connected) con Dropout
class FCLayer(Layer):
    def __init__(self, input_size, output_size, dropout_rate=0.5):
        # np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.dropout_rate = dropout_rate  # Probabilidad de Dropout (para apagar neuronas)

    def forward_propagation(self, input_data, training=True):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias

        if training:
            # Aplicamos Dropout durante el entrenamiento
            self.dropout_mask = (np.random.rand(*self.output.shape) > self.dropout_rate).astype(int)
            self.output *= self.dropout_mask  # Apagamos neuronas aleatoriamente
        else:
            # Durante la inferencia, escalamos el output por la tasa de retención (1 - dropout_rate)
            self.output *= (1 - self.dropout_rate)

        return self.output

    def backward_propagation(self, output_error, learning_rate):
        # Aplicamos el dropout mask también en la propagación hacia atrás
        output_error *= self.dropout_mask  # Solo propagamos el error en las neuronas activas

        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # Actualizar los parámetros
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

# Clase para Capa de Activación. Junto con la capa densa forman perceptrones.
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data, training=True):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

# Clase Red, conecta múltiples capas.
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        if input_data.ndim == 1:  # YA NO SIEMPRE VAMOS A RECIBIR ARREGLOS UNIDIMENSIONALES
            input_data = np.array([[x] for x in input_data])
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation (sin dropout durante la predicción)
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output, training=False)
            result.append(output)
        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        if x_train[0].ndim == 1:  # YA NO SIEMPRE VAMOS A RECIBIR ARREGLOS UNIDIMENSIONALES
            x_train = np.array([[x] for x in x_train])
        samples = len(x_train)
        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation (con dropout durante el entrenamiento)
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output, training=True)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples

            # Para usar en clasificación (con más de dos clases)
            # calculamos el error promedio entre nodos de salida.
            err = np.mean(err)

            # Imprimimos el error promedio de cada época, más que nada
            # para seguimiento del aprendizaje.
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

# Funciones de Activación y sus correspondientes derivadas
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    sig = sigmoid(x)  # Calculamos sigmoid(x) para cada elemento del vector
    return sig * (1 - sig)

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)

# Funciones de pérdida y sus derivadas.
def mse(y_true, y_hat):
    return (y_true - y_hat) ** 2

def mse_prime(y_true, y_hat):
    return 2 * (y_hat - y_true)

def bce(y_true, y_hat):
    return np.mean(-y_true * np.log(y_hat) - (1 - y_true) * np.log(1 - y_hat))

def bce_prime(y_true, y_hat):
    return ((1 - y_true) / (1 - y_hat) - y_true / y_hat) / np.size(y_true)


In [19]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size, dropout_rate=0.5):
        #np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.dropout_rate = dropout_rate  # Probabilidad de Dropout (para apagar neuronas)

    def forward_propagation(self, input_data, training=True):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias

        if training:
            # Aplicamos Dropout durante el entrenamiento
            self.dropout_mask = (np.random.rand(*self.output.shape) > self.dropout_rate).astype(int)
            self.output *= self.dropout_mask  # Apagamos neuronas aleatoriamente
        else:
            # Durante la inferencia, escalamos el output por la tasa de retención (1 - dropout_rate)
            self.output *= (1 - self.dropout_rate)

        return self.output

    def backward_propagation(self, output_error, learning_rate):
        # Aplicamos el dropout mask también en la propagación hacia atrás
        output_error *= self.dropout_mask  # Solo propagamos el error en las neuronas activas

        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # Actualizar los parámetros
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error


In [24]:
def early_stopping(model, X_test, y_test, patience=5, min_delta=0.001):
    best_accuracy = 0
    patience_counter = 0
    for epoch in range(80):  # Ajusta el número de épocas según tus necesidades
        model.fit(X_train, y_train, epochs=1, learning_rate=0.1)  # Entrena el modelo durante una época
        y_hat = model.predict(X_test)  # Predice sobre los datos de prueba
        for i in range(len(y_hat)):
            y_hat[i] = np.argmax(y_hat[i][0])  # Transforma la salida en un vector one-hot encoded
        accuracy = accuracy_score(y_test, y_hat)  # Calcula la exactitud del modelo
        print(f'Epoch {epoch+1}, Accuracy: {accuracy:.4f}')
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping: la exactitud en el conjunto de validación ha dejado de mejorar después de {patience} épocas")
                break
    return model

# ...

# Entrenamos el modelo con datos de entrenamiento y early stopping
model = early_stopping(model, X_test, y_test, patience=5, min_delta=0.001)

# Usamos el modelo para predecir sobre los datos de prueba (validación)
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1.
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

epoch 1/1   error=0.468659
Epoch 1, Accuracy: 0.7070
epoch 1/1   error=0.463874
Epoch 2, Accuracy: 0.7110
epoch 1/1   error=0.458067
Epoch 3, Accuracy: 0.7470
epoch 1/1   error=0.454090
Epoch 4, Accuracy: 0.7530
epoch 1/1   error=0.456410
Epoch 5, Accuracy: 0.7620
epoch 1/1   error=0.454562
Epoch 6, Accuracy: 0.7590
epoch 1/1   error=0.446390
Epoch 7, Accuracy: 0.7670
epoch 1/1   error=0.438992
Epoch 8, Accuracy: 0.7730
epoch 1/1   error=0.441547
Epoch 9, Accuracy: 0.7780
epoch 1/1   error=0.440488
Epoch 10, Accuracy: 0.7820
epoch 1/1   error=0.433677
Epoch 11, Accuracy: 0.7900
epoch 1/1   error=0.434556
Epoch 12, Accuracy: 0.8060
epoch 1/1   error=0.430211
Epoch 13, Accuracy: 0.8060
epoch 1/1   error=0.428439
Epoch 14, Accuracy: 0.7960
epoch 1/1   error=0.425751
Epoch 15, Accuracy: 0.8160
epoch 1/1   error=0.422787
Epoch 16, Accuracy: 0.8210
epoch 1/1   error=0.424841
Epoch 17, Accuracy: 0.8280
epoch 1/1   error=0.424537
Epoch 18, Accuracy: 0.8250
epoch 1/1   error=0.422978
Epoch 19, 

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 128))
model.add(ActivationLayer(relu, relu_prime))
model.add(FCLayer(128, 64))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(64, 10))
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Asignamos función de pérdida
model.use(bce, bce_prime)

# Entrenamos el modelo con datos de entrenamiento
model.fit(X_train, y_train, epochs=80, learning_rate=0.1)

# Usamos el modelo para predecir sobre los datos de prueba (validación)
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1.
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

epoch 1/80   error=0.518734
epoch 2/80   error=0.509886
epoch 3/80   error=0.491296
epoch 4/80   error=0.502741
epoch 5/80   error=0.487247
epoch 6/80   error=0.484225
epoch 7/80   error=0.482278
epoch 8/80   error=0.461884
epoch 9/80   error=0.463521
epoch 10/80   error=0.469897
epoch 11/80   error=0.461972
epoch 12/80   error=0.456526
epoch 13/80   error=0.447594
epoch 14/80   error=0.448014
epoch 15/80   error=0.451160
epoch 16/80   error=0.451539
epoch 17/80   error=0.438532
epoch 18/80   error=0.439232
epoch 19/80   error=0.439925
epoch 20/80   error=0.436278
epoch 21/80   error=0.432609
epoch 22/80   error=0.435254
epoch 23/80   error=0.429870
epoch 24/80   error=0.427978
epoch 25/80   error=0.422648
epoch 26/80   error=0.430734
epoch 27/80   error=0.425504
epoch 28/80   error=0.424259
epoch 29/80   error=0.416432
epoch 30/80   error=0.418383
epoch 31/80   error=0.417529
epoch 32/80   error=0.422138
epoch 33/80   error=0.414185
epoch 34/80   error=0.413822
epoch 35/80   error=0.4

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Necesitamos identificar cuántos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Parámetros para Dropout (probabilidad de apagar neuronas)
dropout_rate = 0.5  # Puedes ajustar este valor entre 0.2 y 0.5

# Agregamos capas al modelo con Dropout
model.add(FCLayer(entrada_dim, 128, dropout_rate=dropout_rate))
model.add(ActivationLayer(relu, relu_prime))
model.add(FCLayer(128, 64, dropout_rate=dropout_rate))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(64, 10, dropout_rate=dropout_rate))
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Asignamos función de pérdida
model.use(bce, bce_prime)

# Entrenamos el modelo con datos de entrenamiento
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre los datos de prueba (validación)
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1.
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf, '\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test, y_hat)))


epoch 1/10   error=0.524829
epoch 2/10   error=0.511097
epoch 3/10   error=0.503990
epoch 4/10   error=0.491889
epoch 5/10   error=0.494010
epoch 6/10   error=0.479365
epoch 7/10   error=0.480733
epoch 8/10   error=0.475314
epoch 9/10   error=0.465396
epoch 10/10   error=0.463378
MATRIZ DE CONFUSIÓN para modelo ANN
[[ 81   0   0   1   0   0   3   0   2   0]
 [  0 116   0   1   0   1   0   0   0   0]
 [ 13   8  38   6   1   3  43   5   3   1]
 [  4   1   1  89   1   2   2   2   6   8]
 [  0   0   0   0 101   0   3   0   1  14]
 [  7   8   0  17   4  28   7   0   0   0]
 [  3   0   1   0   1   1  88   1   0   0]
 [  1   3   0   0   5   0   0  61   3   5]
 [  8   6   0  15   3   3   3   1  60   3]
 [  0   2   0   2  25   0   2   4   0  58]] 

La exactitud de testeo del modelo ANN es: 0.720


In [23]:
# Función para entrenar el modelo usando mini-batches
def train_with_minibatch(model, X_train, y_train, X_test, y_test, batch_size=32, epochs=10, patience=5):
    best_accuracy = 0
    patience_counter = 0
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        # Iterar a través de los datos en mini-batches
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            model.train_on_batch(X_batch, y_batch)
        # Evaluar el modelo al final de cada época
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        print(f'Loss: {loss:.4f}, Accuracy: {acc:.4f}')

        # Early stopping
        if acc > best_accuracy:
            best_accuracy = acc
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping: la precisión en el conjunto de validación ha dejado de mejorar después de {patience} épocas")
                break

# Función principal para manejar ambos tipos de redes (fully connected y cnn)
def main(data_type='fully_connected'):
    # Generar datos de acuerdo al tipo de red
    X, y = generate_data(data_type=data_type)

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Construir el modelo de acuerdo al tipo de datos
    if data_type == 'fully_connected':
        model = build_fully_connected_model()
    elif data_type == 'cnn':
        model = build_cnn_model_with_pooling()

    # Compilar el modelo
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrenar el modelo usando mini-batches
    train_with_minibatch(model, X_train, y_train, X_test, y_test, batch_size=32, epochs=10, patience=5)

# Ejecutar el código para la red Fully Connected
print("Entrenando red Fully Connected con mini-batches:")
main(data_type='fully_connected')

# Ejecutar el código para la red CNN con capas de Pooling
print("\nEntrenando red CNN con capas de Pooling y mini-batches:")
main(data_type='cnn')

Entrenando red Fully Connected con mini-batches:


NameError: name 'generate_data' is not defined